Краулер для Проза ру,

авторы, рекомендованные редакцией

http://www.proza.ru/authors/editor.html

In [1]:
# Импортируем необходимые библиотеки:
import random
import time
import requests # http-запросы,
import re # регулярные выражения,
from bs4 import BeautifulSoup # удаление тегов html,
from tqdm import tqdm # красотуля для анализа прогресса.

In [2]:
#список всех ссылок
hrefs = []
out = open(r"/home/mi_air/Downloads/spisok_proza_hrefs.txt","w", encoding="utf-8")

In [ ]:
# Функия принимает адрес статьи на nplus1.ru и возвращает текст статьи и метаинформацию по ней.
def getArticleTextProza(adr):
    r = requests.get(adr) # Получаем текст статьи, ...
#print(r.text)
    tables=re.split("</div>",re.split('="tables"', r.text)[1])[0] # ... берем блок с метаинформацией, ...
# ... извлекаем из блока метаинформацию. 
    t1=re.split("</time>", re.split("<time", tables)[1])[0]
    n_time=re.split("</span>", re.split("<span>", t1)[1])[0]
    n_date=re.split("</span>", re.split("<span>", t1)[2])[0]
    n_rubr=re.split(">", re.split("</a>", re.split("<a href", tables)[1])[0])[1]
    n_diff=re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
# Достаем заголовок, автора и текст статьи.
    n_head=re.split("</h1>",re.split('<h1>', r.text)[1])[0]
    n_author=re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
    n_text=re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

# "Откусываем" оставшиеся теги.
    beaux_text=BeautifulSoup(n_text, "lxml")
    n_text=beaux_text.get_text() 

# print(n_time, n_date, n_rubr, n_diff)
# print(n_head)
# print(n_author)
# print(n_text)
    return [n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]

# def getArticleTextNPlus1(adr):

In [3]:
# Получаем текст cо страницы, которая содержит ссылки на всех авторов, рекомендуемых сегодня

def getAuthors(link):
    r = requests.get(link)
    # Каждая ссылка на автора оформлена с помощью тега <li><a href="/avtor/
    refs=re.split('<li><a href="/avtor/', r.text)
    database=[]
    
    for i in refs[1:]:
        urlauthor = re.split(" class='recomlink'>",i)[0]
        urlauthor = "http://www.proza.ru/avtor/" + urlauthor
        urlauthor = urlauthor.strip('"')
        hrefs.append(urlauthor)
        out.write(urlauthor+"\n")
        print(urlauthor)
    #return(hrefs)




In [4]:
initial = "http://www.proza.ru/authors/editor.html" #первая ссылка,с кот.начинается обход
link1 = initial
while True:
    spisok = getAuthors(link1)
    s = requests.get(link1)
    refs="http://www.proza.ru/" + re.split('">', re.split('<p>Вы можете посмотреть список авторов, рекомендуемых редакцией <a href="', s.text)[1])[0]
    link1 = refs



http://www.proza.ru/avtor/mudamailru
http://www.proza.ru/avtor/peresvetm
http://www.proza.ru/avtor/oleg51
http://www.proza.ru/avtor/sava22sava
http://www.proza.ru/avtor/stepanych
http://www.proza.ru/avtor/ovolkov
http://www.proza.ru/avtor/arestova
http://www.proza.ru/avtor/shuvcgl
http://www.proza.ru/avtor/dorothea
http://www.proza.ru/avtor/meatland
http://www.proza.ru/avtor/valeri07
http://www.proza.ru/avtor/peresvetm
http://www.proza.ru/avtor/stepanych
http://www.proza.ru/avtor/kseniaerr
http://www.proza.ru/avtor/krasin1
http://www.proza.ru/avtor/verrba
http://www.proza.ru/avtor/sava22sava
http://www.proza.ru/avtor/vitcan5183
http://www.proza.ru/avtor/povelika2
http://www.proza.ru/avtor/dorothea
http://www.proza.ru/avtor/peresvetm
http://www.proza.ru/avtor/vagarin1yandex
http://www.proza.ru/avtor/stepanych
http://www.proza.ru/avtor/lanit
http://www.proza.ru/avtor/alex1957
http://www.proza.ru/avtor/sentebr
http://www.proza.ru/avtor/stepanych
http://www.proza.ru/avtor/pplmi
http://www.

IndexError: list index out of range

In [6]:
#необходимо проверить список на дубли
authors = open(r"/home/mi_air/Downloads/spisok_proza_hrefs.txt", "r", encoding="utf-8")
lst = []
for line in authors:
    lst.append(line)
lst = list(set(lst))


In [7]:
out2 = open(r"/home/mi_air/Downloads/spisok_proza_hrefs_dedup.txt", "w", encoding="utf-8")
for item in lst:
    out2.write(item)

In [8]:
authortext = open(r"/home/mi_air/Downloads/proza_ru/spisok_proza_author_texts.txt", "w", encoding="utf-8")

In [9]:
#Теперь список авторов нам нужно превратить с список данных автора и ссылки на его тексты:
def getAuthorInfo(authorlink):
    r = requests.get(authorlink)
    try:
        author_name = re.split(" / Проза.ру</title>",re.split('<title>', r.text)[1])[0]
        author_items = re.split("</b>", re.split("Произведений: <b>", r.text)[1])[0]
        author_readeres = re.split("</b>", re.split("Читателей</a>: <b>", r.text)[1])[0]
        author_texts_features = []
        texts = re.split("<h2>Произведения</h2>", r.text)[1]
        texts2 = re.split('<li><a href="', texts)
        for line in texts2[1:]:
            if "poemlink" in line:
                textlink = "http://www.proza.ru" + re.split('" class',line)[0]
                textname = re.split("</a>", re.split('poemlink">', line)[1])[0]
                textdate = re.split(" ", re.split(", ", re.split("<small>- ",line)[1])[1])[0]
                textrubric = re.split(",", re.split("<small>- ", line)[1])[0]
                texttime = re.split(".\d{4} ",re.split("</small></li>", line)[0])[1]
                features = [author_name,author_items,author_readeres,textname,textrubric,textdate,texttime,textlink]
                author_texts_features.append(features)
                line = ""
                for i in features:
                    line += str(i)+"\t"
                line += "\n"
                authortext.write(line)
    except:
        pass

    return(author_texts_features )

    

In [10]:
hrefs = open(r"/home/mi_air/Downloads/spisok_proza_hrefs_dedup.txt", "r", encoding="utf-8")
for link in hrefs:
    authorlink = link
    getAuthorInfo(authorlink)
    a = 50
    continuelink = authorlink + "&s=" + str(a)

    while True:    
        try:
            continuelink = authorlink + "&s=" + str(a)
            getAuthorInfo(continuelink)
            a+=50
        except:
            break